In [1]:
from __future__ import print_function

import pandas as pd
import os 
import boto3
from datetime import datetime
ts = datetime.now().strftime("%Y-%m-%d")
    
root_folder = os.getcwd()
root_folder = "/home/paperspace/data"
print(root_folder)
image_root = '/home/paperspace/data/images/niihau'
# local_jpg_path = "/home/paperspace/data/images/niihau/"+os.path.basename(row.s3_key)

/home/paperspace/data


In [2]:
# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

In [3]:
import boto3 
boto3.setup_default_session(profile_name='hawaii')
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')
bucket_name = 'hawaii-marine-debris'
version='v3'

In [4]:
import boto3
import pandas as pd

test_annotations_key = "ml/{}/data/keras_annotations_test.csv".format(version)
local_test_annotation_path = "{}/keras_annotations_test.csv".format(root_folder)
s3_resource.Bucket(bucket_name).download_file(test_annotations_key, local_test_annotation_path)

# obj = s3_client.get_object(Bucket=bucket_name, Key=test_annotations_key)
test_annotation_df = pd.read_csv(local_test_annotation_path, encoding='utf-8',names=['image_path','x0','y0','x1','y1','label'])
test_annotation_df[:5]


annotations_key = "ml/{}/data/keras_annotations_train.csv".format(version)
annotation_path = "{}/keras_annotations_train_{}.csv".format(root_folder,version)
s3_resource.Bucket(bucket_name).download_file(annotations_key, annotation_path)

# obj = s3_client.get_object(Bucket=bucket_name, Key=test_annotations_key)
annotation_df = pd.read_csv(annotation_path, encoding='utf-8',names=['image_path','x0','y0','x1','y1','label'])
annotation_df = annotation_df.append(test_annotation_df)



In [5]:
len(annotation_df)
# test_annotation_df = annotation_df

20409

In [6]:

if 'image_path' in annotation_df.columns:
    annotation_df['s3_key'] = annotation_df['image_path'].apply(lambda s: s.replace(image_root,'niihau/jpg')).apply(lambda s: s.replace("/home/paperspace/data/images/niihau",'niihau/jpg'))
annotation_df[:3]

,image_path,x0,y0,x1,y1,label,s3_key
0,/home/paperspace/data/images/niihau/2_720_03_0...,308,474,358,524,Foam,niihau/jpg/2_720_03_01.jpg
1,/home/paperspace/data/images/niihau/2_1544_35_...,562,291,862,591,Net,niihau/jpg/2_1544_35_13.jpg
2,/home/paperspace/data/images/niihau/2_1699_22_...,215,680,265,730,Buoys,niihau/jpg/2_1699_22_04.jpg


In [7]:
if 'image_path' in test_annotation_df.columns:
    test_annotation_df['s3_key'] = test_annotation_df['image_path'].apply(lambda s: s.replace(image_root,'niihau/jpg')).apply(lambda s: s.replace("/home/paperspace/data/images/niihau",'niihau/jpg'))
if version!='v1':
    test_annotation_df['label'] = test_annotation_df['label'].replace({'B':'Buoys','C': 'Cloth','F':'Foam','L':'Line','M':'Metal','N':'Net','P':'Plastic','T':'Tire','W':'wood','V':'Boat'})
test_annotation_df.groupby('label').size().reset_index(name='counts')


,label,counts
0,Boat,1
1,Buoys,187
2,Foam,26
3,Line,18
4,Metal,4
5,Net,75
6,Other,57
7,Plastic,356
8,Processedwood,14
9,Tire,36


In [8]:
## with negative images

len(test_annotation_df[test_annotation_df["label"].isnull()])

1237

In [9]:
test_annotation_df = test_annotation_df.sort_values('s3_key')
test_annotation_df[:1]

,image_path,x0,y0,x1,y1,label,s3_key
1373,/home/paperspace/data/images/niihau/1_1006_27_...,0,0,0,0,NaN,niihau/jpg/1_1006_27_11.jpg


In [10]:
first_row = test_annotation_df.iloc[0]
local_image_path = "{}/{}".format(root_folder, os.path.basename(first_row['s3_key']))
print(first_row['s3_key'])


niihau/jpg/1_1006_27_11.jpg


In [11]:
# show images inline
%matplotlib inline


# import keras
import keras
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

Using TensorFlow backend.


In [12]:

def display_objects(image_file , actual=[], threshold = 0.1):
    image = read_image_bgr(image_file)
    
    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(40, 40))

    # preprocess image for network
    image = preprocess_image(image)
    # image, scale = resize_image(image)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    print("processing time: ", time.time() - start)
    print("detected {} labels".format(len(labels[0])))
    
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < threshold :
            print("not enough confidence")
            break
        
        color = label_color(label)
        print(box)
        b = box.astype(int)
        draw_box(draw, b, color=color)
        if score >= threshold:
            caption = "{} {:.3f}".format(label_lookup[label], score)
            print(caption)
        else:
            caption = ""
        draw_caption(draw, b, caption)
    for obj in actual:
        draw_box(draw, obj, color=[31  , 0   , 255])
    plt.figure(figsize=(120, 120))

    plt.imshow(draw)
    plt.show()
    
def detect_objects(image_file, threshold = 0.1):
    image = read_image_bgr(image_file)
    detected_objects = []
    image = preprocess_image(image)
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    max_items = 10
    
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < threshold:
            break
        b = box.astype(int)
        detected_objects.append({'x1':b[0], 'y1':b[1], 'x2': b[2], 'y2':b[3],'label':label,'score':score})
        if len(detected_objects)>=max_items:
            break
    return detected_objects
    
def detect_debris(row):
    local_image_path = "{}/{}".format(root_folder, os.path.basename(first_row['s3_key']))
    if not os.path.isfile(local_image_path):
        s3_resource.Bucket(bucket_name).download_file(row['s3_key'], local_image_path)
    display_objects(local_image_path)

In [13]:
def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    assert bb1['x1'] <= bb1['x2']
    assert bb1['y1'] <= bb1['y2']
    assert bb2['x1'] <= bb2['x2']
    assert bb2['y1'] <= bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

#bb1: predicted
#bb2: actual
def overlapped_type(bb1, bb2):
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']
    
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])
    
    if x_left==bb1['x1'] and y_top==bb1['y1'] and x_right==bb1['x2'] and y_bottom==bb1['y2']:
        return 'outside'
    elif x_left==bb2['x1'] and y_top==bb2['y1'] and x_right==bb2['x2'] and y_bottom==bb2['y2']:
        return 'within'
    else:
        return 'overwrapped'

    

In [14]:
model_path = "/tmp/model_{}_10_6.h5".format(version)
print(model_path)
model_key = "ml/model/debris_model_{}_10_6.h5".format(version)
print(model_key)
if not os.path.isfile(model_path):
    s3_resource.Bucket(bucket_name).download_file(model_key, model_path)

model = models.load_model(model_path, backbone_name='resnet50')


/tmp/model_v3_10_6.h5
ml/model/debris_model_v3_10_6.h5


/home/paperspace/miniconda3/envs/gpu/lib/python3.6/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [15]:
label_path = "/tmp/labels_{}.csv".format(version)
s3_resource.Bucket(bucket_name).download_file('ml/{}/data/labels.csv'.format(version), label_path)
label_df = pd.read_csv(label_path,names=['label','id'])
label_df
label_lookup = label_df.set_index('id').T.to_dict('records')[0]
label_lookup 

{0: 'Bicycle',
 1: 'Boat',
 2: 'Surfboard',
 3: 'Bottle',
 4: 'Plastic bag',
 5: 'Wheel',
 6: 'Tire',
 7: 'Buoys',
 8: 'Cloth',
 9: 'Foam',
 10: 'Line',
 11: 'Metal',
 12: 'Net',
 13: 'Plastic',
 14: 'Processedwood',
 15: 'Other'}

In [16]:
test_annotation_df['label_found'] = None
test_annotation_df['wrong_label'] = None
test_annotation_df['detected'] = None
test_annotation_df['detected_bounding_type'] = None


In [17]:
test_annotation_df.columns


Index(['image_path', 'x0', 'y0', 'x1', 'y1', 'label', 's3_key', 'label_found',
       'wrong_label', 'detected', 'detected_bounding_type'],
      dtype='object')

In [18]:
#quick test with 1 test data set
detection_per_image = {}

row = test_annotation_df.iloc[0]
x1, y1, x2, y2, label = row['x0'], row['y0'], row['x1'], row['y1'], row['label'] #42,347,849,1134
annotation ={'x1':x1, 'x2':x2, 'y1':y1, 'y2':y2, 'label':label}

local_image_path = "{}/{}".format(root_folder, os.path.basename(row['s3_key']))
os.path.isfile(local_image_path)
if not os.path.isfile(row.image_path) and not os.path.isfile(local_image_path):
    print("downloading image")
    s3_resource.Bucket(bucket_name).download_file(row['s3_key'], local_image_path)
print("image ready",end='\r')


In [19]:
len(test_annotation_df)

2011

In [20]:
#calculate recall : how many did we get it and missed?

In [21]:
num_found = 0
processed = 0
wrong_label = 0

# needs to evaluate these
new_detection =0 

# deteobjs = detect_objects(local_image_path)
all_debris_detected = []
for i, row in test_annotation_df.iterrows():
    x1, y1, x2, y2, label = row['x0'], row['y0'], row['x1'], row['y1'], row['label'] #42,347,849,1134
    annotation ={'x1':x1, 'x2':x2, 'y1':y1, 'y2':y2, 'label':label}
    local_image_path = "{}/{}".format(image_root, os.path.basename(row['s3_key']))

    #download, if it's not there yet.
    if not os.path.isfile(row.image_path) and not os.path.isfile(local_image_path):
        s3_resource.Bucket(bucket_name).download_file(row['s3_key'], local_image_path)
        print("image ready",end='\r')

    if not local_image_path in detection_per_image:
        deteobjs = detect_objects(local_image_path)
        detection_per_image[local_image_path] = deteobjs
        print("Detected objects",end='\r')
    else:
        deteobjs = detection_per_image[local_image_path]
    found = False
    for obj in deteobjs:
        overlap_ratio =get_iou(obj,annotation)
        if(overlap_ratio):
            found = True
            num_found +=1
            test_annotation_df.loc[i, 'detected_bounding_type'] = overlapped_type(obj,annotation)
            test_annotation_df.loc[i, 'detected'] = str(obj)
            if version == "v1":
                test_annotation_df.loc[i, 'label_found'] = label_lookup[str(obj['label'])]
            else:
                test_annotation_df.loc[i, 'label_found'] = label_lookup[obj['label']]
            obj['overlapped_with_test_index'] = i
        if obj['score'] >0.15:
            obj['s3_key']=row['s3_key']
            all_debris_detected.append(obj)
        if obj['score'] <0.1:
            break
    processed +=1
    message =("%s debris found out of %s" % (num_found, processed))
    print(message,end='\r')
    

In [22]:
test_annotation_df

,image_path,x0,y0,x1,y1,label,s3_key,label_found,wrong_label,detected,detected_bounding_type
1373,/home/paperspace/data/images/niihau/1_1006_27_...,0,0,0,0,NaN,niihau/jpg/1_1006_27_11.jpg,None,None,None,None
1144,/home/paperspace/data/images/niihau/1_1006_27_...,0,0,0,0,NaN,niihau/jpg/1_1006_27_28.jpg,None,None,None,None
800,/home/paperspace/data/images/niihau/1_1007_02_...,0,0,0,0,NaN,niihau/jpg/1_1007_02_31.jpg,None,None,None,None
1473,/home/paperspace/data/images/niihau/1_1007_14_...,0,0,0,0,NaN,niihau/jpg/1_1007_14_09.jpg,None,None,None,None
1558,/home/paperspace/data/images/niihau/1_1007_24_...,0,0,0,0,NaN,niihau/jpg/1_1007_24_16.jpg,None,None,None,None
803,/home/paperspace/data/images/niihau/1_1007_24_...,0,0,0,0,NaN,niihau/jpg/1_1007_24_27.jpg,None,None,None,None
1986,/home/paperspace/data/images/niihau/1_1007_32_...,0,0,0,0,NaN,niihau/jpg/1_1007_32_09.jpg,None,None,None,None
915,/home/paperspace/data/images/niihau/1_1008_02_...,0,0,0,0,NaN,niihau/jpg/1_1008_02_13.jpg,None,None,None,None
1139,/home/paperspace/data/images/niihau/1_1008_03_...,0,0,0,0,NaN,niihau/jpg/1_1008_03_17.jpg,None,None,None,None
1130,/home/paperspace/data/images/niihau/1_1008_05_...,0,0,0,0,NaN,niihau/jpg/1_1008_05_08.jpg,None,None,None,None


In [23]:
all_debris_detected[:3]

[{'x1': 142,
  'y1': 151,
  'x2': 181,
  'y2': 189,
  'label': 13,
  'score': 0.20814364,
  'overlapped_with_test_index': 254,
  's3_key': 'niihau/jpg/1_1048_14_28.jpg'},
 {'x1': 140,
  'y1': 152,
  'x2': 180,
  'y2': 191,
  'label': 12,
  'score': 0.2035287,
  'overlapped_with_test_index': 254,
  's3_key': 'niihau/jpg/1_1048_14_28.jpg'},
 {'x1': 140,
  'y1': 139,
  'x2': 217,
  'y2': 217,
  'label': 13,
  'score': 0.19152547,
  'overlapped_with_test_index': 254,
  's3_key': 'niihau/jpg/1_1048_14_28.jpg'}]

In [24]:
# now calculate precision : how many of detected were correct?
print("%s debris found out of %s" % (num_found, processed))
len(all_debris_detected)


1384 debris found out of 2011


7095

In [ ]:
#check against training data also. maybe we can use this as new annotations
num_found = 0
processed = 0
# needs to evaluate these
new_detection =0 


##refactor this later

target_df = annotation_df

for i, row in target_df.iterrows():
    x1, y1, x2, y2, label = row['x0'], row['y0'], row['x1'], row['y1'], row['label'] #42,347,849,1134
    annotation ={'x1':x1, 'x2':x2, 'y1':y1, 'y2':y2, 'label':label}
    local_image_path = "{}/{}".format(image_root, os.path.basename(row['s3_key']))

    #download, if it's not there yet.
    if not os.path.isfile(row.image_path) and not os.path.isfile(local_image_path):
        s3_resource.Bucket(bucket_name).download_file(row['s3_key'], local_image_path)
        print("image ready",end='\r')

    if not local_image_path in detection_per_image:
        deteobjs = detect_objects(local_image_path)
        detection_per_image[local_image_path] = deteobjs
        print("Detected objects",end='\r')
    else:
        deteobjs = detection_per_image[local_image_path]
    
    found = False
    
    for obj in deteobjs:
        overlap_ratio =get_iou(obj,annotation)
        if(overlap_ratio):
            found = True
            num_found +=1
            target_df.loc[i, 'detected_bounding_type'] = overlapped_type(obj,annotation)
            target_df.loc[i, 'detected'] = str(obj)
            if version == "v1":
                target_df.loc[i, 'label_found'] = label_lookup[str(obj['label'])]
            else:
                target_df.loc[i, 'label_found'] = label_lookup[obj['label']]
            obj['overlapped_with_training_index'] = i
        if obj['score'] >0.1:
            obj['s3_key']=row['s3_key']
            all_debris_detected.append(obj)
        if obj['score'] <=0.1:
            break
    processed +=1
    message =("%s debris found out of %s" % (num_found, processed))
    print(message,end='\r')
    
#expect these to be really high accuracy. needs evalate for those it missed

In [ ]:
print("%s debris found out of %s and detected %s " % (num_found, processed, len(all_debris_detected)))



In [ ]:
#json_to_csv

import csv
f = csv.writer(open('all_debris_detected.csv', "w+"))
f.writerow(["image_s3_key","x1", "y1", "x2", "y2", "label","score","overlapped_test_index",'overlapped_with_training_index'])

for x in all_debris_detected:
    val1, val2="",""
    if "overlapped_with_test_index" in x:
        val1=x["overlapped_with_test_index"]
    if "overlapped_with_training_index" in x:
        val2=x["overlapped_with_training_index"]
    f.writerow([x["s3_key"],
                x["x1"],
                x["y1"],
                x["x2"],
                x["y2"],
                x["label"],
                x["score"],
                val1,val2])

In [ ]:
all_detected = pd.read_csv('all_debris_detected.csv')
print(len(all_detected))
deduped_all_detected = all_detected.drop_duplicates()
len(deduped_all_detected)

In [ ]:

deduped_all_detected.to_csv("all_debris_detected_with_all.csv")

In [ ]:
s3_client.upload_file('all_debris_detected_with_all.csv', bucket_name, 'ml/result/{}/all_debris_detected_with_all.csv'.format(version))


In [ ]:
import pickle
pik_filepath = "detection_per_image.pkl"

with open(pik_filepath, "wb") as f:
    pickle.dump(detection_per_image, f)


In [34]:
#to open it
with open('data.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    data = pickle.load(f)
    pickle.load("detection_per_image.pkl")

TypeError: file must have 'read' and 'readline' attributes

In [ ]:
####TODO:::

## out of those we detected that we didn't have it before, upload to x to verify if it's the real debris.



In [82]:
!head all_debris_detected.csv

version| recall   | precision | notes |
-------|----------|-----------|-------|
v1     | 101/678  | 101/636   | without OID|
v2     | 203/694  | 203/598   | with OID | 
v3     | 183/648  | 183/714   | with negative image and more type of debris|
v3.10_6| 417/648  | 417/676   | 7 epochs with 5000 steps to avoid overfitting | 

In [ ]:
s3_client.upload_file('all_debris_detected.csv', bucket_name, 'ml/result/{}/all_debris_detected_2.csv'.format(version))


# test_annotation_df

In [ ]:
# print(str(obj))
# test_annotation_df.loc[1, 'detected'] = str(obj)
test_annotation_df[test_annotation_df['detected_bounding_type'].isin(['within','outside','overwrapped'])].to_csv('bounding_box_detected_{}.csv'.format(version), encoding='utf-8')
s3_client.upload_file('bounding_box_detected_{}.csv'.format(version), bucket_name, 'ml/result/{}_bounding_box_detected_{}.csv'.format(version, ts))


In [ ]:
test_annotation_df.to_csv("evaluation_result.csv", encoding='utf-8')
s3_client.upload_file('evaluation_result.csv', bucket_name, 'ml/result/evaluation_result_all_{}_{}.csv'.format(version, ts))


In [ ]:
# all_debris_detected.to_csv("all_debris_detected.csv",encoding='utf-8')
# s3_client.upload_file('all_debris_detected.csv', bucket_name, 'ml/result/{}/all_debris_detected.csv'.format(v))


In [ ]:
len(test_annotation_df)

In [ ]:
len(test_annotation_df[~test_annotation_df.index.duplicated()])

In [ ]:
test_annotation_df= test_annotation_df.reset_index()

In [ ]:
import numpy as np

# test_annotation_df.iloc[0]

annotated_df = test_annotation_df[~test_annotation_df["label"].isnull()]
non_annotated_df = test_annotation_df[test_annotation_df["label"].isnull()]

In [ ]:
len(annotated_df)

In [ ]:

annotated_df.groupby('label').size().reset_index(name='counts')
#wrong detection
wrong_labeled = annotated_df[test_annotation_df['label_found'].notnull()][annotated_df['label_found']!=annotated_df['label']]
correct_labled = annotated_df[test_annotation_df['label_found'].notnull()][annotated_df['label_found']==annotated_df['label']]

stats = {
    "wrong_label":len(wrong_labeled),
    "correct_label":len(correct_labled),
    "missed":len(annotated_df[annotated_df['label_found'].isna()]),
}
stats

In [ ]:
correct_labled.groupby('label').size().reset_index(name='counts')


In [ ]:
wrong_labeled.groupby('label').size().reset_index(name='counts')
